In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

d:\ana\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# New instruction dataset
guanaco_dataset = "trip-data"

# Fine-tuned model
new_model = "llama-2-7b-chat-trip"

In [5]:
import pandas as pd

df = pd.read_parquet("hf://datasets/yananchen/natural_plan__trip_planning/data/test-00000-of-00001.parquet")

In [8]:
df

,num_cities,cities,durations,prompt_0shot,prompt_5shot,golden_plan,pred_5shot_pro,id
0,3,Helsinki**Barcelona**Florence,5**5**6,You plan to visit 3 European cities for 14 day...,You are an expert at planning trips. You are g...,Here is the trip plan for visiting the 3 Europ...,Here is the trip plan for visiting the 3 Europ...,trip_planning_example_0
1,3,Riga**Manchester**Split,7**4**6,You plan to visit 3 European cities for 15 day...,You are an expert at planning trips. You are g...,Here is the trip plan for visiting the 3 Europ...,Here is the trip plan for visiting the 3 Europ...,trip_planning_example_1
2,3,Reykjavik**Vienna**Venice,2**7**4,You plan to visit 3 European cities for 11 day...,You are an expert at planning trips. You are g...,Here is the trip plan for visiting the 3 Europ...,Here is the trip plan for visiting the 3 Europ...,trip_planning_example_2
3,3,Berlin**Warsaw**Bucharest,3**3**2,You plan to visit 3 European cities for 6 days...,You are an expert at planning trips. You are g...,Here is the trip plan for visiting the 3 Europ...,Here is the trip plan for visiting the 3 Europ...,trip_planning_example_3
4,3,Seville**Munich**Tallinn,7**5**2,You plan to visit 3 European cities for 12 day...,You are an expert at planning trips. You are g...,Here is the trip plan for visiting the 3 Europ...,Here is the trip plan for visiting the 3 Europ...,trip_planning_example_4
...,...,...,...,...,...,...,...,...
1595,10,Manchester**Santorini**Bucharest**Lyon**Valenc...,4**4**2**2**3**5**2**4**4**5,You plan to visit 10 European cities for 26 da...,You are an expert at planning trips. You are g...,Here is the trip plan for visiting the 10 Euro...,Here is the trip plan for visiting the 10 Euro...,trip_planning_example_1595
1596,10,Edinburgh**Barcelona**Budapest**Vienna**Stockh...,5**5**5**5**2**3**2**5**5**4,You plan to visit 10 European cities for 32 da...,You are an expert at planning trips. You are g...,Here is the trip plan for visiting the 10 Euro...,Here is the trip plan for visiting the 10 Euro...,trip_planning_example_1596
1597,10,Lyon**Oslo**Milan**Warsaw**Amsterdam**Florence...,3**3**2**4**4**3**2**2**3**2,You plan to visit 10 European cities for 19 da...,You are an expert at planning trips. You are g...,Here is the trip plan for visiting the 10 Euro...,Here is the trip plan for visiting the 10 Euro...,trip_planning_example_1597
1598,10,Berlin**Stockholm**Santorini**Athens**Prague**...,3**2**5**3**3**5**2**3**4**5,You plan to visit 10 European cities for 26 da...,You are an expert at planning trips. You are g...,Here is the trip plan for visiting the 10 Euro...,Here is the trip plan for visiting the 10 Euro...,trip_planning_example_1598


In [7]:
df.to_csv('trip.csv', index=False)